In [ ]:
!pip install -q -U datasets
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

df2=pd.read_excel("/content/Telecom Churn Rate Dataset.xlsx")
df2=df2.head(100)



In [ ]:
df2.shape

(100, 23)

In [ ]:
#df2.drop(columns=["ICD-10 Code"],inplace=True)

In [ ]:
df2 = df2[['customerID', 'PhoneService', 'MonthlyCharges', 'TotalCharges']]

In [ ]:
df2["output"]=df2.apply(lambda row:"CUSTOMERID:" + row['customerID'] + "  PHONE SERVICE:" + row['PhoneService'] + "  MONTHLY CHARGES:" + str(row['MonthlyCharges']) + "  TOTAL CHARGES:" + str(row['TotalCharges'])+ "  ->:Extracted entities "+"CUSTOMERID:" + row['customerID'] + "  PHONE SERVICE:" + row['PhoneService'] + "  MONTHLY CHARGES:" + str(row['MonthlyCharges']) + "  TOTAL CHARGES:" + str(row['TotalCharges']), axis = 1)

In [ ]:
df2["output"]

0     CUSTOMERID:7590-VHVEG  PHONE SERVICE:No  MONTH...
1     CUSTOMERID:5575-GNVDE  PHONE SERVICE:Yes  MONT...
2     CUSTOMERID:3668-QPYBK  PHONE SERVICE:Yes  MONT...
3     CUSTOMERID:7795-CFOCW  PHONE SERVICE:No  MONTH...
4     CUSTOMERID:9237-HQITU  PHONE SERVICE:Yes  MONT...
                            ...                        
95    CUSTOMERID:8637-XJIVR  PHONE SERVICE:Yes  MONT...
96    CUSTOMERID:9803-FTJCG  PHONE SERVICE:Yes  MONT...
97    CUSTOMERID:0278-YXOOG  PHONE SERVICE:Yes  MONT...
98    CUSTOMERID:3212-KXOCR  PHONE SERVICE:Yes  MONT...
99    CUSTOMERID:4598-XLKNJ  PHONE SERVICE:Yes  MONT...
Name: output, Length: 100, dtype: object

In [ ]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(df2),
})

In [ ]:
train_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['customerID', 'PhoneService', 'MonthlyCharges', 'TotalCharges', 'output'],
        num_rows: 100
    })
})

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    TextStreamer
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "Deep/finetuned-llama"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
df2["output"][2]

'CUSTOMERID:3668-QPYBK  PHONE SERVICE:Yes  MONTHLY CHARGES:53.85  TOTAL CHARGES:108.15  ->:Extracted entities CUSTOMERID:3668-QPYBK  PHONE SERVICE:Yes  MONTHLY CHARGES:53.85  TOTAL CHARGES:108.15'

In [ ]:
import transformers
pipeline = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200
    # torch_dtype=torch.float16,
    # trust_remote_code=True,
    # device_map="auto",
)


prompt = "Extract values corresponding to the entities as entity value pairs. CUSTOMERID:7590-VHVEG PHONE SERVICE:No MONTHLY CHARGES:29.85 TOTAL CHARGES:29.85."
output = pipeline(f"<s>[INST]{prompt}[\INST]")
print(output[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST]Extract values corresponding to the entities as entity value pairs. CUSTOMERID:7590-VHVEG PHONE SERVICE:No MONTHLY CHARGES:29.85 TOTAL CHARGES:29.85.[\INST]  Sure! Here are the entity value pairs for the given text:

Entities:

* CUSTOMERID: 7590
* PHONE SERVICE: No
* MONTHLY CHARGES: 29.85
* TOTAL CHARGES: 29.85

Value Pairs:

* CUSTOMERID: 7590 - VHVEG
* PHONE SERVICE: No
* MONTHLY CHARGES: 29.85
* TOTAL CHARGES: 


In [ ]:
# lora_alpha = 32
# lora_dropout = 0.1
# lora_r = 4

peft_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    # target_modules=["q_proj","v_proj"]
)

In [ ]:
output_dir = "./results"
num_train_epochs=5
per_device_train_batch_size = 4
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
weight_decay=0.001
fp16=False
bf16=False
max_grad_norm = 0.3
max_steps =100
warmup_ratio = 0.03
lr_scheduler_type = "constant"
report_to="tensorboard"


training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay = weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to = report_to
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    peft_config=peft_config,
    dataset_text_field="output",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.293800
20,0.498800
30,0.425200
40,0.414300
50,0.404700
60,0.386900
70,0.371200
80,0.349200
90,0.335300
100,0.316500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=100, training_loss=0.4796023440361023, metrics={'train_runtime': 350.6459, 'train_samples_per_second': 2.282, 'train_steps_per_second': 0.285, 'total_flos': 1776044852379648.0, 'train_loss': 0.4796023440361023, 'epoch': 8.0})

In [ ]:
trainer.model.save_pretrained(refined_model)

In [ ]:
import transformers
from langchain import HuggingFacePipeline

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
text_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=200
    # trust_remote_code=True,
    # device_map="auto",
)
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

telecom_data = "CUSTOMERID:7590-VHVEG  PHONE SERVICE:No  MONTHLY CHARGES:29.85  TOTAL CHARGES:29.85"


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
prompt_template = "Extract values corresponding to the entities as entity value pairs. {context}"
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain(telecom_data)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'context': 'CUSTOMERID:7590-VHVEG  PHONE SERVICE:No  MONTHLY CHARGES:29.85  TOTAL CHARGES:29.85',
 'text': 'Extract values corresponding to the entities as entity value pairs. CUSTOMERID:7590-VHVEG  PHONE SERVICE:No  MONTHLY CHARGES:29.85  TOTAL CHARGES:29.85  ->:Extracted entities CUSTOMERID:7590-VHVEG  PHONE SERVICE:No  MONTHLY CHARGES:29.85  TOTAL CHARGES:29.85  ->:Entities extracted  Now, you can use the entities to search for related information or to perform further analysis.  For example, you can use the entity "CUSTOMERID:7590-VHVEG" to search for information about the customer or use the entity "PHONE SERVICE:No" to identify customers who do not have phone service. '}

In [ ]:
telecom_data_2 = "CUSTOMERID:5575-GNVDE  PHONE SERVICE:Yes  MONTHLY CHARGES:56.95  TOTAL CHARGES:1889.5 "

In [ ]:
llm_chain(telecom_data_2)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'context': 'CUSTOMERID:5575-GNVDE  PHONE SERVICE:Yes  MONTHLY CHARGES:56.95  TOTAL CHARGES:1889.5 ',
 'text': 'Extract values corresponding to the entities as entity value pairs. CUSTOMERID:5575-GNVDE  PHONE SERVICE:Yes  MONTHLY CHARGES:56.95  TOTAL CHARGES:1889.5  ->:Extracted entities CUSTOMERID:5575-GNVDE  PHONE SERVICE:Yes  MONTHLY CHARGES:56.95  TOTAL CHARGES:1889.5  ->:Entities extracted Now, let\'s move on to the next step, which is to classify the entities into categories. In this case, we can classify the entities into the following categories:  Entities with a category of "Customer Information":  CUSTOMERID:5575-GNVDE  ->:Category of "Customer Information" Entities with'}

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!mkdir deepanshu

In [ ]:
!cp -r "/content/Deep" "/content/drive/MyDrive"

In [ ]:
!cp -r "/content/results" "/content/drive/MyDrive"

In [ ]:
from peft import PeftModel

model_path = "/content/Deep/finetuned-llama"  # saving in colab as gdrive is 15gb storage only

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, '/content/drive/MyDrive/Deep/finetuned-llama')
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
import transformers
from langchain import HuggingFacePipeline

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
text_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=200
    # trust_remote_code=True,
    # device_map="auto",
)
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

telecom_data = "CUSTOMERID:5575-GNVDE  PHONE SERVICE:Yes  MONTHLY CHARGES:56.95  TOTAL CHARGES:1889.5 "


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
prompt_template = "Extract values corresponding to the entities as entity value pairs. {context}"
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain(telecom_data)

In [ ]:
patient_data_2="CUSTOMERID:7590-VHVEG  PHONE SERVICE:No  MONTHLY CHARGES:29.85  TOTAL CHARGES:29.85"

In [ ]:
llm_chain(patient_data_2)

In [ ]:
patient_data_3="CUSTOMERID:3668-QPYBK  PHONE SERVICE:Yes  MONTHLY CHARGES:53.85  TOTAL CHARGES:108.15  "

In [ ]:
llm_chain(patient_data_3)

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi
api=HfApi()
repo_id="dmody1/finetuned-llama"
base_path="/content/Deep/finetuned-llama"
local_file_paths=[base_path + "/special_tokens_map.json",
                  base_path + "/tokenizer_config.json",
                  base_path + "/tokenizer.json",
                  base_path + "/generation_config.json",
                  base_path + "/config.json",
                  base_path + "/pytorch_model-00001-of-00002.bin",
                  base_path + "/pytorch_model-00002-of-00002.bin",
                  base_path + "/pytorch_model.bin.index.json",
                  base_path + "/adapter_config.json",
                  base_path + "/adapter_model.bin",
                  base_path + "/README.md"]
for local_file_path in local_file_paths:
  file_name=local_file_path.split("/")[-1]
  path_in_repo=file_name
  api.upload_file(
      path_or_fileobj=local_file_path,
      path_in_repo=path_in_repo,
      repo_id=repo_id,
      repo_type="model"

  )
  print(f"Uploaded {file_name} to {repo_id}")